In [ ]:
# coding: utf8

In [7]:
###
# https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA
###

#Import & Definition

In [81]:
import sys
import os
import csv
import pandas as pd

In [82]:
DATA_DIR = u'/home/lmdadm/data/db/'
FILE = u'accidentologie.csv'
TARGET = u'https://maps.googleapis.com/maps/api/geocode/json?address='

# Utils

In [133]:
class crossair:
    """ Match the road, build an url, request to google and fill the couchbase"""
    
    def __init__(self, path, bucket, t):
        import pprint
        
        self.source = path
        self.bucket = bucket
        self.stock = []
        self.target = t
        self.p = pprint.PrettyPrinter(indent=2)
        #self.df = pd.read_csv(self.source, delimiter=';', encoding='utf_8')
        with open(self.source) as csv:
            for line in csv:
                row = line.split(';')
                self.stock.append(row[4])
        
    
    def _req(self, street):
        """ just convert the street 99, AVENUE NIEL
            In a valid url: https://maps.googleapis.com/maps/api/geocode/json?address=99,+AVENUE+NIEL+PARIS
            String street: ...
        """ 
        street = street.strip()
        if(street[0] == ','):
            street = street[1:]
        
        query = street.replace(' ', '+')
        #self.p.pprint(query)
        return self.target + query + '+PARIS'
    
    def rollUp(self):
        import urllib2
        import json
        err = []
        urls = []
        
        for s in self.stock:
            res = urllib2.urlopen(self._req(s)).read()
            doc = json.loads(res)
            flag = doc.pop('status')
            if(flag == 'OK'):
                del doc['results'][0]['address_components']
                self.insert(doc)
                print('')
            else:
                pass
        self.p.pprint(flag)
        
    def insert(self, doc):
        print(doc)
        

In [134]:
cr = crossair(DATA_DIR + FILE,'Geocoding', TARGET)

In [135]:
cr.rollUp()

u'OK'


In [86]:
cr._req('37, RUE DE RIVOLI')

'37,+RUE+DE+RIVOLI'


u'https://maps.googleapis.com/maps/api/geocode/json?address=37,+RUE+DE+RIVOLI'